In [362]:
# Imports necessary modules
from suds.client import Client
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import datetime

# Loads series indexes for lookup
series_bcb = pd.read_csv('~/Downloads/cotacoes.txt', index_col=0, header=0)
# Lookup expression
#   series_bcb[series_bcb['Descrição'].str.contains("Dólar")==True]

# User input
print('Acesso à API do Banco Central do Brasil (v. 0.1)\n')
series_selecionadas = list(int(x) for x in input("Quais séries deseja recuperar (espaço entre as séries): ").split()) or [1, 21619]
datainicio = str(input("Data inicial (DD/MM/AAAA): ") or '01/09/2014')
datafim = str(input("Data final (DD/MM/AAAA): ") or '01/09/2016')
nome_output = input("Nome do arquivo de saída: ") or 'BCB_SGS'

# Activates WSDL and retrieves time series
url = 'https://www3.bcb.gov.br/sgspub/JSP/sgsgeral/FachadaWSSGS.wsdl'
bcb = Client(url)
try:
    dados = bcb.service.getValoresSeriesXML(series_selecionadas, datainicio, datafim)
except Exception as detail:
    print(detail)

# Parse XML data
root = ET.fromstring(dados)

# Creates Pandas DataFrame with calandar days as index
index = pd.date_range(datetime.datetime.strptime(datainicio, '%d/%m/%Y').strftime('%m/%d/%y'), datetime.datetime.strptime(datafim, '%d/%m/%Y').strftime('%m/%d/%y'))
df = pd.DataFrame(index=index, columns=series_selecionadas)

# Updates the DataFrame values
for serie in range(len(root)):
    for ponto in range(len(root[serie])):
            df.set_value(datetime.datetime.strptime(root[serie][ponto][0].text, '%d/%m/%Y').strftime('%m/%d/%y'), int(root[serie].attrib['ID']), float(root[serie][ponto][1].text))
df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))

# Previous list generator
#   datas = []
#   for serie in range(len(root)):
#       for ponto in range(len(root[serie])):
#              datas.append([int(root[serie].attrib['ID']), root[serie][ponto][0].text, float(root[serie][ponto][1].text)])
# Equivalent comprehension
#   [(int(root[serie].attrib['ID']), root[serie][ponto][0].text, float(root[serie][ponto][1].text)) for serie in range(len(root)) for ponto in range(len(root[serie]))]

# Export to CSV file
series_selecionadas_nom = {}
for x in series_selecionadas:
    series_selecionadas_nom[x] = series_bcb.get_value(x, 'Descrição')
df = df.dropna()
df = df.rename(columns=series_selecionadas_nom)
df.to_csv('~/Desktop/'+nome_output+'.csv')

Acesso à API do Banco Central do Brasil (v. 0.1)

Quais séries deseja recuperar (espaço entre as séries): 
Data inicial (DD/MM/AAAA): 
Data final (DD/MM/AAAA): 
Nome do arquivo de saída: 
